# Project Operations

In [1]:
%load_ext literary.notebook

In [2]:
from pathlib import Path

from traitlets import List, Unicode, default, observe, validate
from traitlets.config import Config, LoggingConfigurable

from .config import find_project_config
from .trait import Path as PathTrait

In [3]:
class ProjectOperator(LoggingConfigurable):
    project_dir = PathTrait(help="Path to Literary project top-level directory").tag(
        config=True
    )

    packages_dir = Unicode(
        "src", help="Path to Literary packages top-level directory"
    ).tag(config=True)

    @validate("project_dir")
    def _validate_project_dir(self, proposal):
        path = Path(proposal.value)
        if not path.exists():
            raise ValueError()
        return path

    @classmethod
    def config_for_project_dir(cls, project_dir: str) -> Config:
        """Return the Config object that configures this Configurable for
        the given project directory.

        :param project_dir: directory to project
        """
        return Config({cls.__name__: {"project_dir": project_dir}})

    @property
    def packages_path(self) -> Path:
        return self.resolve_path(self.packages_dir)

    def resolve_path(self, path):
        return self.project_dir / path